In [1]:
from methods import *
from model import LogisticInstance
import numpy as np
from cvxopt import matrix, solvers
import time

solvers.options['show_progress'] = False

In [2]:
def test(method_name, method, real_f_name, real_f, **kwargs):
    t = time.monotonic()
    x_opt = method(**kwargs)
    print('{}:\n  x_opt = {}\n  {} = {}\n  time = {}s\n'.format(method_name, x_opt,
                                                                real_f_name, real_f(x_opt),
                                                                time.monotonic() - t))

### Simple 1D case

\begin{align*}
\frac{e^{x-2}(5-x)}{1 + e^{x-2}} &\to max \\
x & \le 2
\end{align*}

Theoretical answer is $x_{opt} \sim 2.44$ in unconditional case and $x_{opt} = 2$ otherwise.

In [3]:
a = np.array([1.])
b = -2.
c = np.array([-1.])
d = 5.
F = np.array([[1.]])
g = np.array([2.])

issue = LogisticInstance(a, b, c, d, F, g)

In [4]:
# unconditional optimization

f = lambda x: -issue.log_expected_profit(x)
df = lambda x: -issue.dlog_expected_profit(x)
d2f = lambda x: -issue.d2log_expected_profit(x)



test('Gradient descent, conv. by argument, h_k = 0.1', 
     GradientDescent(h=0.1), 
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

test('Gradient descent, conv. by argument, h_k = 1/k', 
     GradientDescent(h=lambda step: 1. / step), 
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

test('Gradient descent, conv. by argument, h_k = 1/k', 
     GradientDescent(h=lambda step: 1. / step,
                     convergence_condition=Convergence.ByValue), 
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

test('Gradient descent, conv. by argument, h_k = 1/sqrt(k)', 
     GradientDescent(h=lambda step: step ** -0.5), 
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

print('\n')

test('Accelerated Nesterov gradient descent, h_k = 0.5', 
     AcceleratedNesterovGradientDescent(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

test('Accelerated Nesterov gradient descent, h_k = 1/sqrt(k)', 
     AcceleratedNesterovGradientDescent(h=lambda step: step ** -0.5),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

print('\n')

test('Newton method',
     NewtonMethod(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.array([1.]), df=df, d2f=d2f)

test('Broyden-Fletcher-Goldfarb-Shanno method (quasy-Newton)', 
     BroydenFletcherGoldfarbShannoMethod(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.array([1.]), df=df)

Gradient descent, conv. by argument, h_k = 0.1:
  x_opt = [2.44261703]
  expected_profit = 1.5571455818420734
  time = 0.011573543000849895s

Gradient descent, conv. by argument, h_k = 1/k:
  x_opt = [2.34713717]
  expected_profit = 1.554373820931351
  time = 0.1462936419993639s

Gradient descent, conv. by argument, h_k = 1/k:
  x_opt = [2.24594916]
  expected_profit = 1.545516050628967
  time = 0.018999893000000156s

Gradient descent, conv. by argument, h_k = 1/sqrt(k):
  x_opt = [2.44255797]
  expected_profit = 1.5571455722441265
  time = 0.004981541002052836s



Accelerated Nesterov gradient descent, h_k = 0.5:
  x_opt = [2.4427758]
  expected_profit = 1.5571455971164652
  time = 0.0015145889992709272s

Accelerated Nesterov gradient descent, h_k = 1/sqrt(k):
  x_opt = [2.44126018]
  expected_profit = 1.5571448252478892
  time = 0.0018778400008159224s



Newton method:
  x_opt = [2.44285568]
  expected_profit = 1.5571455989971115
  time = 0.0006504790035251062s

Broyden-Fletcher-Gold

In [5]:
# conditional

g = lambda x: np.dot(issue.F , x) -issue.g
dg = lambda x: issue.F

test('''Penalty method, pen_k = [1] * (k-1), ext_pen_func = max(0, x)^0.7,
unconditional method -- gradient descent with h_k=0.01''', 
     PenaltyMethod(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df,
     g=g, dg=dg)

test('Conditional gradient method, k = 2 / (3 + step)',
     ConditionalGradientMethod(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df,
     g=g)

test('Conditional gradient method, k = 1 / sqrt(step)',
     ConditionalGradientMethod(k=lambda step: 1. / np.sqrt(step)),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df,
     g=g)

Penalty method, pen_k = [1] * (k-1), ext_pen_func = max(0, x)^0.7,
unconditional method -- gradient descent with h_k=0.01:
  x_opt = [2.00233491]
  expected_profit = 1.500582363040146
  time = 0.06820202099697781s

Conditional gradient method, k = 2 / (3 + step):
  x_opt = [1.99783784]
  expected_profit = 1.4994582913553598
  time = 0.039937362002092414s

Conditional gradient method, k = 1 / sqrt(step):
  x_opt = [2.]
  expected_profit = 1.5
  time = 0.0006944110027689021s



In [6]:
# linear

A = matrix(-issue.a)
B = matrix([issue.b])
F = matrix(issue.F)
G = matrix(issue.g)


sol = solvers.lp(A, F, G)
print('  x_opt = ', np.array(sol['x']).T[0], '\n   prob = ', issue.prob(np.array(sol['x']).T[0]))

  x_opt =  [2.] 
   prob =  0.5


### More complex 2D case

x = (price, amount of advertising, quality of packing)

In [10]:
a = np.array([-4, 2, 1])
b = 0.
c = np.array([4000., -40., -4.])
d = -10.
F = np.array([-c, np.array([0., -1., 0.]), np.array([0., 0., -1.])])
g = np.array([d - 100., 0., 0.])

issue = LogisticInstance(a, b, c, d, F, g)

f = lambda x: -issue.log_expected_profit(x)
df = lambda x: -issue.dlog_expected_profit(x)
d2f = lambda x: -issue.d2log_expected_profit(x)
g = lambda x: np.dot(issue.F , x) -issue.g
dg = lambda x: issue.F

In [11]:
test('''Conditional gradient''', 
     ConditionalGradientMethod(precision=1e-3),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.array([1, 1, 1]), df=df,
     g=g, dg=dg)

Conditional gradient:
  x_opt = [  2.19556097 190.75708493 190.72844573]
  expected_profit = 379.04670443815667
  time = 0.06694302299729316s

